In [5]:
import numpy as np
import os
import matplotlib.pyplot as plt
import librosa
from dtaidistance import dtw
from fastdtw import fastdtw
from scipy.signal import argrelextrema

In [6]:
segment_dir = 'segment'
pair_dir = 'pair'

In [58]:
visited = []

for segmentx_name in os.listdir(segment_dir):
    # Skip if already matched
    if segmentx_name in visited:
        continue

    # Get anchor
    x = np.load(os.path.join(segment_dir, segmentx_name))

    # Create folder for set
    os.makedirs(os.path.join(pair_dir, segmentx_name[:-4]), exist_ok=True)

    for segmenty_name in os.listdir(segment_dir):
        pair = True
        # Skip if already matched
        if segmenty_name in visited:
            continue

        # Load candidate
        y = np.load(os.path.join(segment_dir, segmenty_name))

        # Allow reasonable length difference
        if abs(len(x) - len(y)) > 4:
            continue

        # Allow reasonable difference in standard deviations
        if abs(np.std(x) - np.std(y)) > 10:
            continue
        
        for i in range(0, min(len(y), len(x)) - 3, 3):
            xi = x[i:i+3]
            yi = y[i:i+3]
            distance, paths = fastdtw(xi, yi)

            if distance > 100:
                pair = False
                break

        if not pair:
            continue

        visited.append(segmenty_name)

        # Save the segments and plots in the set folder
        plt.plot(x)
        plt.plot(y)
        plt.savefig(f'{pair_dir}/{segmentx_name[:-4]}/{segmenty_name[:-4]}.png')
        plt.close()
        np.save(f'{pair_dir}/{segmentx_name[:-4]}/{segmenty_name}', y)

    
    plt.show()